## Import all necessary Frameworks

In [1]:
import pandas as pd
import requests
from datetime import datetime
import holidays
import hopsworks

## Specify timeline and frequency

In [2]:
START_DATE = "2019-12-31"
END_DATE = "2025-12-27"
dates = pd.date_range(START_DATE, END_DATE, freq="D")

## Function for fetching weather data 
* This function uses meteo api to fetch the data
* We use the weather data in stockholm
* We collect history data about snowfall, wind, rain, temperature
* returns a dataframe with the features 

In [3]:
def fetch_weather(start_date, end_date):
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": 59.6519, # Stockholm 
        "longitude": 17.9186,
        "start_date": start_date,
        "end_date": end_date,
        "daily": [
            "temperature_2m_mean",
            "precipitation_sum",
            "snowfall_sum",
            "windspeed_10m_max"
        ],
        "timezone": "Europe/Stockholm"
    }

    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()["daily"]

    df = pd.DataFrame(data)
    df.rename(columns={
        "time": "date",
        "temperature_2m_mean": "tavg",
        "precipitation_sum": "prcp",
        "snowfall_sum": "snow",
        "windspeed_10m_max": "wspd"
    }, inplace=True)

    df["date"] = pd.to_datetime(df["date"])
    return df

## Fetching holiday and calender data in Sweden 
* We want to data about holiday and seasons
* All of the values are transformed into bigint 

In [4]:
dates

DatetimeIndex(['2019-12-31', '2020-01-01', '2020-01-02', '2020-01-03',
               '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09',
               ...
               '2025-12-18', '2025-12-19', '2025-12-20', '2025-12-21',
               '2025-12-22', '2025-12-23', '2025-12-24', '2025-12-25',
               '2025-12-26', '2025-12-27'],
              dtype='datetime64[ns]', length=2189, freq='D')

In [6]:
calendar = pd.DataFrame({"date": dates})

years = calendar["date"].dt.year.unique().tolist()
se_holidays = holidays.Sweden(years=years)


calendar["day_of_week"] = calendar["date"].dt.weekday.astype(int)
calendar["is_weekend"] = calendar["day_of_week"].isin([5, 6]).astype(int)
calendar["week_of_year"] = calendar["date"].dt.isocalendar().week.astype(int)
calendar["month"] = calendar["date"].dt.month.astype(int)
calendar["is_holiday"] = calendar["date"].isin(se_holidays).astype(int)

In [7]:
weather = fetch_weather(START_DATE, END_DATE)

features = (
    calendar
    .merge(weather, on="date", how="left")
    .sort_values("date")
    .ffill()
)

features 

,date,day_of_week,is_weekend,week_of_year,month,is_holiday,tavg,prcp,snow,wspd
0,2019-12-31,1,0,1,12,0,2.8,0.0,0.00,22.0
1,2020-01-01,2,0,1,1,1,2.3,0.1,0.00,24.3
2,2020-01-02,3,0,1,1,0,4.9,0.0,0.00,28.3
3,2020-01-03,4,0,1,1,0,5.2,0.0,0.00,31.3
4,2020-01-04,5,1,1,1,0,1.5,0.3,0.21,25.7
...,...,...,...,...,...,...,...,...,...,...
2184,2025-12-23,1,0,52,12,0,-2.6,0.0,0.00,12.6
2185,2025-12-24,2,0,52,12,0,-4.0,0.0,0.00,12.3
2186,2025-12-25,3,0,52,12,1,-0.3,0.0,0.00,20.7
2187,2025-12-26,4,0,52,12,1,1.3,0.0,0.00,16.8


## Upload the data to Hopsworks

In [8]:
project = hopsworks.login()
fs = project.get_feature_store()

fg = fs.get_or_create_feature_group(
    name="stockholm_weather_calendar_features",
    version=1,
    primary_key=["date"],
    event_time="date",
    description="Daily Stockholm weather + calendar features"
)

fg.insert(features, write_options={"wait_for_job": True})

2026-01-11 18:52:28,039 INFO: Initializing external client
2026-01-11 18:52:28,040 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-11 18:52:30,001 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296539
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1296539/fs/1284172/fg/1938769


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████████| Rows 2189/2189 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: stockholm_weather_calendar_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296539/jobs/named/stockholm_weather_calendar_features_1_offline_fg_materialization/executions
2026-01-11 18:52:49,458 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-11 18:52:55,861 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-11 18:54:41,334 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-11 18:54:41,526 INFO: Waiting for log aggregation to finish.
2026-01-11 18:54:50,215 INFO: Execution finished successfully.


(Job('stockholm_weather_calendar_features_1_offline_fg_materialization', 'SPARK'),
 None)